In [ ]:
%pip install ultralytics tqdm numpy opencv-python datasets

In [1]:
import os
import json
import re
import random
import string

In [2]:

import cv2
from ultralytics import YOLO
import shutil
import numpy as np
import logging
from tqdm import tqdm


# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [3]:
current_working_directory = os.getcwd()

YOLO_path = os.path.join(current_working_directory, "models/yolov8n-pose.pt") #path to the YOLO model
datasetDir = os.path.join(current_working_directory, "archive", "Demo") #change here to the other dataset like RFC-2000
newKeypointsDir = os.path.join(current_working_directory, "archive", "Keypoints-Demo") #The new json data are saved here
os.makedirs(newKeypointsDir, exist_ok=True)

In [9]:
# Options for saving
save_original = True  # Set to True if you want to save the original video
save_processed = True  # Set to True if you want to save the processed video with keypoints

In [5]:

model = YOLO(YOLO_path) #staring the model

In [6]:
# Define dataset categories and video processing limits
sets = ['']
categories = ['Fight', 'NonFight']
video_limit = 10  # Limit number of videos processed per category

#making the labels in the COCO format for YOLO model
labeledKeypoints = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear",
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

In [7]:
# Helper function to sanitize filenames
def sanitize_filename(filename):
    # Remove any character that is not a letter, number, or underscore
    sanitized_name = re.sub(r'[^A-Za-z0-9_]', '', filename)
    
    # If the sanitized name is too short or empty, create a random name
    if len(sanitized_name) < 5:
        sanitized_name = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
    
    return sanitized_name

In [11]:
# Process each set (train/val) and category (Fight/NonFight)
for set_name in tqdm(sets, desc="Processing Sets"):
    for category in tqdm(categories, desc=f"Processing Categories in {set_name}", leave=False):
        input_dir = os.path.join(datasetDir, set_name, category)
        video_count = 0

        for video_file in tqdm(os.listdir(input_dir), desc=f"Processing Videos in {category}", leave=False):
            if not video_file.endswith(('.avi', '.mp4')):
                continue
            
            ## uncomment the code below to apply the video limit
            # if video_count >= video_limit:
            #     break

            # Sanitize or generate a unique name for problematic video filenames
            video_name = sanitize_filename(os.path.splitext(video_file)[0])
            video_dir = os.path.join(newKeypointsDir, set_name, category, f"{video_name}")
            os.makedirs(video_dir, exist_ok=True)

            video_path = os.path.join(input_dir, video_file)
            json_output_path = os.path.join(video_dir, f"{video_name}.json")

            # Copy original video if save_original is True
            if save_original:
                original_save_path = os.path.join(video_dir, f"{video_name}.avi")
                try:
                    shutil.copy2(video_path, original_save_path)
                    logging.info(f"Original video saved: {original_save_path}")
                except FileNotFoundError as e:
                    logging.warning(f"File not found or accessible: {video_path}. Skipping this file.")
                    continue

            video_capture = cv2.VideoCapture(video_path)
            frame_index = 0
            keyPointsData = []

            # Set up writer for processed video if save_processed is True
            if save_processed:
                processed_video_path = os.path.join(video_dir, f"{video_name}_processed.avi")
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                fps = video_capture.get(cv2.CAP_PROP_FPS)
                frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
                frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
                processed_video_writer = cv2.VideoWriter(processed_video_path, fourcc, fps, (frame_width, frame_height))
                logging.info(f"Processed video writer initialized for: {processed_video_path}")

            # Process each frame
            while video_capture.isOpened():
                ret, frame = video_capture.read()
                if not ret:
                    break

                # Detect keypoints using YOLO model
                results = model(frame)
                newFrameData = []

                # Extract keypoints and bounding box data for JSON output
                for result in results:
                    boxes = result.boxes
                    keypoints = result.keypoints

                    if boxes is not None and len(boxes) > 0:
                        for i in range(len(boxes)):
                            box_data = boxes.xyxy[i].cpu().numpy()
                            confidence = boxes.conf[i].cpu().item()
                            box = {
                                "x1": float(box_data[0]),
                                "y1": float(box_data[1]),
                                "x2": float(box_data[2]),
                                "y2": float(box_data[3])
                            }

                            keypoints_data = []
                            if keypoints is not None:
                                keypoints_array = keypoints.data[i].cpu().numpy()
                                for j, (x, y, conf) in enumerate(keypoints_array):
                                    keypoints_data.append({
                                        "label": labeledKeypoints[j],
                                        "coordinates": {"x": float(x), "y": float(y)},
                                        "confidence": float(conf)
                                    })

                            newFrameData.append({
                                "person_id": i + 1,
                                "confidence": confidence,
                                "box": box,
                                "keypoints": keypoints_data
                            })

                keyPointsData.append({"frame": frame_index, "detections": newFrameData})

                # Use results[0].plot() for processed video frame
                if save_processed:
                    frame_with_keypoints = results[0].plot()
                    processed_video_writer.write(frame_with_keypoints)

                frame_index += 1

            # Release resources
            video_capture.release()
            if save_processed:
                processed_video_writer.release()
                logging.info(f"Processed video saved to: {processed_video_path}")
                
            # Save JSON data for keypoints
            with open(json_output_path, 'w') as json_file:
                json.dump(keyPointsData, json_file, indent=4)
                logging.info(f"Keypoints data saved to: {json_output_path}")

            video_count += 1

print("Done")


Processing Sets:   0%|          | 0/1 [00:00<?, ?it/s]
2024-11-21 12:25:09,910 - INFO - Original video saved: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_00_Pro\WIN_20241121_11_46_00_Pro.avi


2024-11-21 12:25:10,360 - INFO - Processed video writer initialized for: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_00_Pro\WIN_20241121_11_46_00_Pro_processed.avi



0: 384x640 2 persons, 133.1ms
Speed: 6.3ms preprocess, 133.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 121.5ms
Speed: 2.2ms preprocess, 121.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 177.5ms
Speed: 0.0ms preprocess, 177.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 239.9ms
Speed: 5.0ms preprocess, 239.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 254.8ms
Speed: 4.8ms preprocess, 254.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 231.5ms
Speed: 9.3ms preprocess, 231.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 246.4ms
Speed: 7.0ms preprocess, 246.4ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 283.5ms
Speed: 3.0ms preprocess, 283.5ms inference, 2.2ms postprocess per

2024-11-21 12:26:51,872 - INFO - Processed video saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_00_Pro\WIN_20241121_11_46_00_Pro_processed.avi
2024-11-21 12:26:52,361 - INFO - Keypoints data saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_00_Pro\WIN_20241121_11_46_00_Pro.json

2024-11-21 12:26:52,393 - INFO - Original video saved: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_27_Pro\WIN_20241121_11_46_27_Pro.avi
2024-11-21 12:26:52,918 - INFO - Processed video writer initialized for: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\Fight\WIN_20241121_11_46_27_Pro\WIN_20241121_11_46_27_Pro_processed.avi



0: 384x640 2 persons, 235.7ms
Speed: 10.8ms preprocess, 235.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 378.3ms
Speed: 0.0ms preprocess, 378.3ms inference, 10.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 728.0ms
Speed: 22.1ms preprocess, 728.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1617.2ms
Speed: 206.2ms preprocess, 1617.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 1591.9ms
Speed: 10.4ms preprocess, 1591.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 273.2ms
Speed: 0.0ms preprocess, 273.2ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 245.0ms
Speed: 12.1ms preprocess, 245.0ms inference, 9.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 195.8ms
Speed: 4.0ms preprocess, 195.8ms inference, 9.9ms postp

...


0: 384x640 2 persons, 140.1ms
Speed: 4.0ms preprocess, 140.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 138.8ms
Speed: 3.8ms preprocess, 138.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 206.2ms
Speed: 0.0ms preprocess, 206.2ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 177.9ms
Speed: 3.9ms preprocess, 177.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 189.9ms
Speed: 3.6ms preprocess, 189.9ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 179.9ms
Speed: 0.0ms preprocess, 179.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 185.6ms
Speed: 4.0ms preprocess, 185.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 174.0ms
Speed: 7.6ms preprocess, 174.0ms inference, 0.0ms postprocess per 

2024-11-21 12:36:10,035 - INFO - Processed video saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_15_42_Pro\WIN_20241121_12_15_42_Pro_processed.avi
2024-11-21 12:36:10,377 - INFO - Keypoints data saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_15_42_Pro\WIN_20241121_12_15_42_Pro.json

2024-11-21 12:36:10,419 - INFO - Original video saved: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_18_04_Pro\WIN_20241121_12_18_04_Pro.avi
2024-11-21 12:36:10,622 - INFO - Processed video writer initialized for: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_18_04_Pro\WIN_20241121_12_18_04_Pro_processed.avi



0: 384x640 3 persons, 154.4ms
Speed: 5.0ms preprocess, 154.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 177.5ms
Speed: 2.1ms preprocess, 177.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 140.5ms
Speed: 2.5ms preprocess, 140.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 142.1ms
Speed: 2.2ms preprocess, 142.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 139.5ms
Speed: 5.5ms preprocess, 139.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 185.7ms
Speed: 2.7ms preprocess, 185.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 175.2ms
Speed: 2.8ms preprocess, 175.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 186.6ms
Speed: 8.0ms preprocess, 186.6ms inference, 0.0ms postprocess per 

2024-11-21 12:38:13,394 - INFO - Processed video saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_18_04_Pro\WIN_20241121_12_18_04_Pro_processed.avi
2024-11-21 12:38:14,073 - INFO - Keypoints data saved to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\Keypoints-Demo\NonFight\WIN_20241121_12_18_04_Pro\WIN_20241121_12_18_04_Pro.json


Processing Sets: 100%|██████████| 1/1 [13:04<00:00, 784.21s/it]

Done
